(sec-dask-dataframe-indexing)=
# 索引

In [1]:
%config InlineBackend.figure_format = 'svg'
import os
import urllib
import shutil
from zipfile import ZipFile

import dask
import dask.dataframe as dd
import pandas as pd
from dask.distributed import LocalCluster, Client

cluster = LocalCluster()
client = Client(cluster)

如 {numref}`fig-pandas-dataframe-model` 所示，pandas DataFrame 主要对二维的表进行处理，有列标签和行标签。行标签通常会被用户忽视，但实际上起着至关重要的作用，比如索引（Indexing）。大多数 pandas DataFrame 的行标签是排好序的索引，比如从 0 开始递增。 DataFrame 里面的数据也是有序的。

```{figure} ../img/ch-dask-dataframe/dataframe-model.svg
---
width: 200px
name: fig-pandas-dataframe-model
---
pandas DataFrame 数据模型
```

创建 pandas DataFrame 时，会在最左侧自动生成了索引列，它不是 DataFrame 的“官方”字段，因为索引列并没有列名。

In [15]:
df = pd.DataFrame({
   'A': ['foo', 'bar', 'baz', 'qux'],
   'B': ['one', 'one', 'two', 'three'],
   'C': [1, 2, 3, 4],
   'D': [10, 20, 30, 40]
})
df

,A,B,C,D
0,foo,one,1,10
1,bar,one,2,20
2,baz,two,3,30
3,qux,three,4,40


也可以设置一个字段作为索引列：

In [19]:
df = df.set_index('A')
df

,B,C,D
A,,,
foo,one,1,10
bar,one,2,20
baz,two,3,30
qux,three,4,40


或者重置回原来的结构：

In [20]:
df = df.reset_index()
df

,A,B,C,D
0,foo,one,1,10
1,bar,one,2,20
2,baz,two,3,30
3,qux,three,4,40


## 有序行索引

Dask DataFrame 由多个 pandas DataFrame 组成，但如何在全局维度维护整个 Dask DataFrame 行标签和行顺序是一个很大的挑战。Dask DataFrame 并没有刻意保留全局有序性，也使得它无法支持所有 pandas DataFrame 的功能。

如 {numref}`fig-dask-dataframe-divisions` 所示，Dask DataFrame 在切分时有 `divisions`。 

```{figure} ../img/ch-dask-dataframe/divisions.svg
---
width: 400px
name: fig-dask-dataframe-divisions
---
Dask DataFrame 的 `divisions`
```

以 Dask 提供的样例数据函数 `dask.datasets.timeseries` 为例，它生成了时间序列，使用时间戳作为行标签，每个 Partition 的边界都被记录下来，存储在 `.divisions` 里。`len(divisons)` 等于 `npartitions + 1`。

In [2]:
ts_df = dask.datasets.timeseries("2018-01-01", "2023-01-01")
print(f"df.npartitions: {ts_df.npartitions}")
print(f"df.divisions: {len(ts_df.divisions)}")

df.npartitions: 1826
df.divisions: 1827


Dask DataFrame 没有记录每个 Partition 中有多少行，因此无法在全局角度支持基于行索引的操作，比如 `iloc`。

In [3]:
try:
    ts_df.iloc[3].compute()
except Exception as e:
    print(f"{type(e).__name__}, {e}")

NotImplementedError, 'DataFrame.iloc' only supports selecting columns. It must be used like 'df.iloc[:, column_indexer]'.


但是可以支持列标签，或者 `:` 这样的通配符：

In [4]:
ts_df.iloc[:, [1, 2]].compute()

,id,x
timestamp,,
2018-01-01 00:00:00,984,0.660595
2018-01-01 00:00:01,960,-0.747564
2018-01-01 00:00:02,1039,0.777117
2018-01-01 00:00:03,1038,-0.501949
2018-01-01 00:00:04,992,0.767979
...,...,...
2022-12-31 23:59:55,1005,-0.102774
2022-12-31 23:59:56,1040,-0.648857
2022-12-31 23:59:57,1019,-0.310174


对于 CSV 文件，Dask DataFrame 并没有自动生成 `divisions`。

In [5]:
folder_path = os.path.join(os.getcwd(), "../data/")
download_url = "https://dp.godaai.org/nyc-flights.zip"
zip_file_path = os.path.join(folder_path, "nyc-flights.zip")
if not os.path.exists(os.path.join(folder_path, "nyc-flights")):
    with urllib.request.urlopen(download_url) as response, open(zip_file_path, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
        zf = ZipFile(zip_file_path, 'r')
        zf.extractall(folder_path)
        zf.close()
file_path = os.path.join(folder_path, "nyc-flights", "*.csv")
flights_ddf = dd.read_csv(file_path,
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': object,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})
flights_ddf.divisions

(None, None, None, None, None, None, None)

因为没有记录每个 Partition 有多少条数据，Dask DataFrame 无法很好地支持一些操作，比如 `median()` 这样的百分位操作，因为这些操作需要：(1) 对数据排序；(2) 定位到特定的行。

In [6]:
try:
    flights_ddf['DepDelay'].median()
except Exception as e:
    print(f"{type(e).__name__}, {e}")

NotImplementedError, Dask doesn't implement an exact median in all cases as this is hard to do in parallel. See the `median_approximate` method instead, which uses an approximate algorithm.


## 设置索引列

### `set_index()`

在 Dask DataFrame 中，我们可以使用 `set_index()` 方法手动设置某一列为索引列，这个操作除了设置某个字段为索引列，还会根据这个字段对全局数据进行排序，它打乱了原来每个 Partition 的数据排序，因此会有很高的成本。

下面的例子展示了 `set_index()` 带来的变化：

In [7]:
def print_partitions(ddf):
    for i in range(ddf.npartitions):
        print(ddf.partitions[i].compute())

df = pd.DataFrame(
    {"col1": ["01", "05", "02", "03", "04"], "col2": ["a", "b", "c", "d", "e"]}
)
ddf = dd.from_pandas(df, npartitions=2)
print_partitions(ddf)

  col1 col2
0   01    a
1   05    b
2   02    c
  col1 col2
3   03    d
4   04    e


In [8]:
ddf2 = ddf.set_index("col1")
print_partitions(ddf2)

     col2
col1     
01      a
     col2
col1     
02      c
03      d
04      e
05      b


这个例子设置 `col1` 列为索引列，2 个 Partition 中的数据被打乱重排。如果是在数据量很大的场景，全局数据排序和重分布的成本极高。因此应该尽量避免这个操作。`set_index()` 也有它的优势，它可以加速下游的计算。

回到时间序列数据，该数据使用时间戳作为索引列。下面使用了两种方式对这份数据 `set_index()`。第一种没有设置 `divisions`，第二种设置了 `divisions`。

第一种不设置 `divisions` 耗时很长，因为 Dask DataFrame 计算了所有 Partiton 的数据分布，并根据分布重排列了所有的 Partition，可以看到，Partition 的数目也发生了变化。

In [9]:
%%time
ts_df1 = ts_df.set_index("id")
nu =  ts_df1.loc[[1001]].name.nunique().compute()
print(f"before set_index npartitions: {ts_df.npartitions}")
print(f"after set_index npartitions: {ts_df1.npartitions}")

before set_index npartitions: 1826
after set_index npartitions: 163
CPU times: user 6.1 s, sys: 3.47 s, total: 9.57 s
Wall time: 19.6 s


第二种方式先提前获取了 `divisions`，然后将这些 `divisions` 用于设置 `set_index()`。设定 `division` 的 `set_index()` 速度更快。

In [10]:
dask_computed_divisions = ts_df.set_index("id").divisions
unique_divisions = list(dict.fromkeys(list(dask_computed_divisions)))

In [11]:
%%time
ts_df2 = ts_df.set_index("id", divisions=unique_divisions)
nuids = ts_df2.loc[[1001]].name.nunique().compute()

CPU times: user 3.25 s, sys: 1.09 s, total: 4.34 s
Wall time: 11.7 s


如果不设置索引列，直接对 `id` 列进行查询，发现反而更快。

In [12]:
%%time
nu = ts_df.loc[ts_df["id"] == 1001].name.nunique().compute()

CPU times: user 1.94 s, sys: 743 ms, total: 2.68 s
Wall time: 8.18 s


所以 Dask DataFrame 要慎重使用 `set_index()`，如果 `set_index()` 之后有很多以下操作，可以考虑使用 `set_index()`。

* 使用 `loc` 对索引列进行过滤
* 两个 Dask DataFrame 在索引列上合并（`merge()`）
* 在索引列上进行分组聚合（`groupby()`）

### `reset_index()`

在 pandas 中，默认 `as_index=True` 时，分组字段经过 `groupby()` 之后成为索引列。索引列在 DataFrame 中并不是正式的数据列，如果分组聚合之后只有一个字段（不考虑分组字段），分组聚合的结果就成了一个 `Series`。比如下面 pandas 的例子，`Origin` 列就是分组字段，如果不设置 `as_index=False`，`groupby("Origin", as_index=False)["DepDelay"].mean()` 生成的是一个 `Series`。

In [13]:
# pandas
file_path = os.path.join(folder_path, "nyc-flights", "1991.csv")
pdf = pd.read_csv(file_path,
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': object,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})
uncancelled_pdf = pdf[pdf["Cancelled"] == False]
avg_pdf = uncancelled_pdf.groupby("Origin", as_index=False)["DepDelay"].mean()
avg_pdf.columns = ["Origin", "AvgDepDelay"]
avg_pdf.sort_values("AvgDepDelay")

,Origin,AvgDepDelay
2,LGA,5.726304
0,EWR,6.916220
1,JFK,9.311532


或者是 `reset_index()`，来取消索引列，分组字段会成为 `DataFrame` 的一个正式的字段。

In [22]:
avg_pdf = uncancelled_pdf.groupby("Origin")["DepDelay"].mean().reset_index()
avg_pdf.columns = ["Origin", "AvgDepDelay"]
avg_pdf.sort_values("AvgDepDelay")

,Origin,AvgDepDelay
2,LGA,5.726304
0,EWR,6.916220
1,JFK,9.311532


Dask DataFrame 的 `groupby()` 不支持 `as_index` 参数。Dask DataFrame 只能使用 `reset_index()` 来取消索引列。

In [14]:
uncancelled_ddf = flights_ddf[flights_ddf["Cancelled"] == False]
avg_ddf = uncancelled_ddf.groupby("Origin")["DepDelay"].mean().reset_index()
avg_ddf.columns = ["Origin", "AvgDepDelay"]
avg_ddf = avg_ddf.compute()
# pandas 只使用了一年数据，因此结果不一样
avg_ddf.sort_values("AvgDepDelay")

,Origin,AvgDepDelay
2,LGA,6.944939
0,EWR,9.997188
1,JFK,10.766914


In [ ]:
client.shutdown()